In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv, find_dotenv
envs = load_dotenv(find_dotenv(), override=True)

import time
import sys
import os
sys.path.append('../')

In [4]:
from deepeval import evaluate
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

from src.database.database_utils import get_weaviate_client
from src.database.weaviate_interface_v4 import WeaviateWCS
from src.llm.llm_interface import LLM
from src.llm.llm_utils import get_token_count, load_azure_openai
from src.llm.prompt_templates import question_answering_prompt_series, huberman_system_message
from src.app_functions import generate_prompt_series

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from datasets import Dataset
from litellm import ModelResponse

import asyncio
import nest_asyncio
nest_asyncio.apply()
from rich import print

2024-11-10 18:01:03.598 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [5]:
# answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.7, model='gpt-4', strict_mode=True)
# test_case = LLMTestCase(
#     input="What if these shoes don't fit?",
#     # Replace this with the actual output from your LLM application
#     actual_output="We offer a 30-day full refund at no extra costs.",
#     retrieval_context=["All customers are eligible for a 30 day full refund at no extra costs."]
# )
# evaluate([test_case], [answer_relevancy_metric], run_async=False, ignore_errors=False)

In [6]:
questions = ["Give a brief explanation of how brain neuroplasticity works",
             "What is the role of dopamine in the body",
             "What is a catecholimine",
             "What does Jocko Willink have to say about leadership",
             "What does Lex Fridman think about the evolution of AI", 
             "How can I support the Huberman Lab podcst",
             "Why do people make self-destructive decisions",
             "Provide a better sleep protocol in list format",
             "What are the topcis that Lex Fridman discusses",
             "Is there a generally positive outlook on the future of AI",
            ]

In [12]:
client = get_weaviate_client()
turbo = LLM(model_name='gpt-4o-mini')
azure = LLM(model_name='gpt-4o')
# azure = load_azure_openai(model_name='gpt-4')
collection_name = 'Huberman_minilm_128'

/Users/matiasweber/.pyenv/versions/3.10.14/lib/python3.10/site-packages/weaviate/warnings.py:130: DeprecationWarning: Dep005: You are using weaviate-client version 4.5.4. The latest version is 4.9.3.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [13]:
def get_answer_bundle(query: str,
                      client: WeaviateWCS,
                      collection_name: str,
                      answer_llm: LLM,
                      ground_truth_llm: LLM=None
                     ) -> tuple[str, list[list[str]], str]:
    '''
    Returns answer, ground truth and associated context from a single query.
    '''
    def format_llm_response(response: ModelResponse) -> str:
        return response.choices[0].message.content

    #1st-stage retrieval (get contexts)
    context = client.hybrid_search(query, collection_name, 
                                   query_properties=['content', 'title', 'summary'],
                                   limit=3, 
                                   return_properties=['content', 'guest', 'summary'])
    #create contexts from content field
    contexts = [d['content'] for d in context]
    
    #generate assistant message prompt
    assist_message = generate_prompt_series(query, context)

    #generate answers from model being evaluated
    answer = format_llm_response(answer_llm.chat_completion(huberman_system_message, assist_message))

    #create ground truth answers
    if ground_truth_llm:
        ground_truth = format_llm_response(ground_truth_llm.chat_completion(huberman_system_message, assist_message))
        return query, contexts, answer, ground_truth
    return query, contexts, answer

In [14]:
from math import ceil
from time import sleep

async def create_test_dataset(questions: list[str], 
                              client: WeaviateWCS,
                              collection_name: str,
                              answer_llm: LLM,
                              ground_truth_llm: LLM=None, 
                              batch_size: int=5, 
                              async_mode: bool=True,
                              disable_internal_tqdm: bool=False):
    total = len(questions)
    progress = tqdm('Queries', total=total, disable=disable_internal_tqdm)
    data = []
    batches = ceil(total/batch_size)
    for i in range(batches):
        batch = questions[i*batch_size:(i+1)*batch_size]
        if async_mode:
            results = await asyncio.gather(*[aget_answer_bundle(query, 
                                                                client, 
                                                                collection_name, 
                                                                answer_llm,
                                                                ground_truth_llm) for query in batch])
            if any(results):
                data.extend(results)
            else:
                raise "No results returned for initial batch, double-check your inputs."
        else:
            with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as executor:
                futures = [executor.submit(get_answer_bundle, query, client, collection_name, answer_llm, ground_truth_llm) for query in batch]
                for future in as_completed(futures):
                    progress.update(1)
                    data.append(future.result())
        print(f"Finished with batch {i+1}, taking a break...")
    
    queries = [d[0] for d in data]
    contexts = [d[1] for d in data]
    answers = [d[2] for d in data]
    dataset = {'queries': queries, 'contexts': contexts, 'answers': answers}
    if len(data[0]) == 4:
        ground_truths = [d[3] for d in data]
        dataset.update(ground_truths=ground_truths)
        return dataset
    return dataset

In [15]:
async def aget_answer_bundle( query: str,
                              client: WeaviateWCS,
                              collection_name: str,
                              answer_llm: LLM,
                              ground_truth_llm: LLM=None
                             ) -> tuple[str, list[list[str]], str]:
    '''
    Returns answer, ground truth and associated context from a single query.
    '''
    #1st-stage retrieval (get contexts)
    context = client.hybrid_search(query, collection_name, 
                                   query_properties=['content', 'title', 'summary'],
                                   limit=3, 
                                   return_properties=['content', 'guest', 'summary'])
    
    #create contexts from content field
    contexts = [d['content'] for d in context]
    
    #generate assistant message prompt
    assist_message = generate_prompt_series(query, context, 2)

    #generate answers from model being evaluated
    answer = await answer_llm.achat_completion(huberman_system_message, assist_message)

    #create ground truth answers
    if ground_truth_llm:
        ground_truth = await ground_truth_llm.achat_completion(huberman_system_message, assist_message)
        return query, contexts, answer, ground_truth
    return query, contexts, answer

In [16]:
data = await create_test_dataset(questions, client, collection_name, turbo, azure)

Finished with batch 1, taking a break...

Finished with batch 2, taking a break...

  0%|                                                                                                 | 0/10 [00:15<?, ?it/s]


In [17]:
def create_eval_dataset(questions: list[str],
                        contexts: list[list[str]],
                        answers: list[str]
                       ) -> EvaluationDataset:
    assert len(questions) == len(contexts) == len(answers), 'Mismatched lengths in input values, retry after correcting'
    test_cases = []
    for i in range(len(questions)):
        test_case = LLMTestCase(input=questions[i],
                                actual_output=answers[i],
                                retrieval_context=contexts[i])
        test_cases.append(test_case)
    return EvaluationDataset(alias='Initial test', test_cases=test_cases)